<a href="https://colab.research.google.com/github/mehrnooshkarimi/GateContrib/blob/master/ICC-Radiomics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅Step 1a: Extract .rar File

In [1]:
from google.colab import files

# This will prompt you to upload your .rar file (e.g., all_objects.rar)
uploaded = files.upload()


Saving static-1.rar to static-1.rar


✅ Step 1b: Extract the .rar File Contents

In [3]:
import os

# List files in the current directory
uploaded_files = os.listdir('/content')
print("Uploaded files:")
for f in uploaded_files:
    print(f)


Uploaded files:
.config
static-1.rar
objects
sample_data


In [4]:
# Install unrar if it's not installed already
!apt-get install unrar -y > /dev/null

# Create a folder to extract into
!mkdir -p /content/objects

# Extract the uploaded RAR file to /content/objects
!unrar x /content/static-1.rar /content/objects/ > /dev/null

# Confirm extraction
import os

extracted_files = os.listdir('/content/objects')
print(f"Extracted {len(extracted_files)} files:")
print(extracted_files[:5])  # Show first 5 file names


Extracted 30 files:
['object5.xlsx', 'object18.xlsx', 'object25.xlsx', 'object17.xlsx', 'object29.xlsx']


✅ Step 1c: Combine All 30 Excel Files into One DataFrame

In [5]:
import pandas as pd
import glob

# Get all Excel files from the extracted folder
file_paths = sorted(glob.glob('/content/objects/object*.xlsx'))

# Create a list to store data from each object
all_data = []

# Loop through each file and tag it with object number
for path in file_paths:
    df = pd.read_excel(path)

    # Extract object number from filename like "object25.xlsx"
    object_id = path.split('/')[-1].replace('object', '').replace('.xlsx', '')
    df['object'] = f'obj_{object_id}'

    all_data.append(df)

# Combine into a single DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# Show info
print(f"Combined shape: {combined_df.shape}")
combined_df.head()


Combined shape: (870, 81)


,id,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,...,glszm_SmallAreaEmphasis,glszm_SmallAreaHighGrayLevelEmphasis,glszm_SmallAreaLowGrayLevelEmphasis,glszm_ZoneEntropy,glszm_ZonePercentage,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Contrast,ngtdm_Strength,object
0,2262,0.219702,0.219702,9.118732,41.505020,37.062922,37.062922,9.659195,37.092012,2330.523763,...,0.526114,18.782815,0.046696,5.099878,0.061983,3.060020,0.003819,0.037373,0.157054,obj_1
1,2263,0.254365,0.254365,9.969012,39.191836,35.123499,35.123499,10.514650,35.398802,2639.546468,...,0.519775,15.438134,0.088980,5.019412,0.040974,3.844340,0.003953,0.052148,0.132029,obj_1
2,2264,0.193572,0.191245,7.631803,39.905951,34.946475,34.946475,8.964189,34.946475,1532.394877,...,0.529748,5.979949,0.104188,3.907938,0.043145,7.062548,0.004302,0.018510,0.072837,obj_1
3,2265,0.220808,0.220808,8.653867,39.191836,34.784058,34.784058,9.201692,35.062027,1980.513753,...,0.437440,14.609541,0.037114,4.422887,0.029142,2.373093,0.004564,0.005908,0.243285,obj_1
4,2266,0.208502,0.208502,8.653867,41.505020,36.741403,36.741403,9.201692,37.004671,2097.543294,...,0.456017,9.594662,0.063866,4.748887,0.041284,5.865187,0.003882,0.025877,0.082058,obj_1


✅ Step 2: Reshape the Data for ICC Calculation

In [6]:
# Reshape from wide to long format
melted = combined_df.melt(
    id_vars=['id', 'object'],
    var_name='feature',
    value_name='value'
)

# Preview the melted data
print(f"Melted shape: {melted.shape}")
melted.head()


Melted shape: (68730, 4)


,id,object,feature,value
0,2262,obj_1,shape_Elongation,0.219702
1,2263,obj_1,shape_Elongation,0.254365
2,2264,obj_1,shape_Elongation,0.193572
3,2265,obj_1,shape_Elongation,0.220808
4,2266,obj_1,shape_Elongation,0.208502


✅ Step 3: Compute ICC (Intraclass Correlation Coefficient) for Each Feature

In [7]:
# Install pingouin for ICC calculation
!pip install pingouin --quiet

import pingouin as pg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 13.0 MB/s eta 0:00:00


✅ Step 4: Calculate ICC(2,1) for Each Feature

In [9]:
icc_results = []

for feature_name, group in melted.groupby('feature'):
    try:
        icc = pg.intraclass_corr(
            data=group,
            targets='id',
            raters='object',
            ratings='value',
            nan_policy='omit'  # <-- ignore missing values
        )

        # Keep only ICC2 row
        icc2_row = icc[icc['Type'] == 'ICC2'].copy()
        icc2_row['feature'] = feature_name
        icc_results.append(icc2_row)

    except Exception as e:
        print(f"Error processing feature {feature_name}: {e}")

# Combine and sort
icc_df = pd.concat(icc_results, ignore_index=True)
icc_df_sorted = icc_df.sort_values(by='ICC', ascending=False)

# Display top stable features
icc_df_sorted[['feature', 'ICC']].head(10)


,feature,ICC
68,shape_MajorAxisLength,0.927771
69,shape_Maximum2DDiameterColumn,0.821229
70,shape_Maximum2DDiameterRow,0.817822
72,shape_Maximum3DDiameter,0.800412
19,glcm_Id,0.664869
20,glcm_Idm,0.656148
47,glrlm_ShortRunEmphasis,0.634406
46,glrlm_RunPercentage,0.628973
27,glcm_JointEntropy,0.624700
17,glcm_DifferenceEntropy,0.616708


✅ Step 5: Save ICC Results to CSV

In [10]:
# Save ICC results to a CSV file
icc_df_sorted.to_csv("icc_results.csv", index=False)

# If you're in Colab, download it
from google.colab import files
files.download("icc_results.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# Save the melted long-format data
melted.to_csv("radiomics_long_data.csv", index=False)

# Download it to your computer
from google.colab import files
files.download("radiomics_long_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
# Merge ICC into the melted data
melted_with_icc = melted.merge(
    icc_df_sorted[['feature', 'ICC']],
    on='feature',
    how='left'
)

# Filter: ICC ≥ 0.9
stable_features_mask = melted_with_icc['ICC'] >= 0.9

# Apply Excel styling with red highlight for stable features
def highlight_stable(row):
    color = 'background-color: red' if row['ICC'] >= 0.9 else ''
    return [color] * len(row)

# Apply styling to all rows
styled = melted_with_icc.style.apply(highlight_stable, axis=1)

# Save to Excel with formatting
styled.to_excel("radiomics_long_with_icc_highlighted.xlsx", index=False, engine='openpyxl')

# Download the Excel file
from google.colab import files
files.download("radiomics_long_with_icc_highlighted.xlsx")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
# Merge ICC into the melted data
melted_with_icc = melted.merge(
    icc_df_sorted[['feature', 'ICC']],
    on='feature',
    how='left'
)

# Define custom highlight function
def highlight_icc_levels(row):
    if pd.isna(row['ICC']):
        return [''] * len(row)
    elif row['ICC'] >= 0.9:
        return ['background-color: red'] * len(row)
    elif row['ICC'] >= 0.7:
        return ['background-color: yellow'] * len(row)
    else:
        return [''] * len(row)

# Apply styling
styled = melted_with_icc.style.apply(highlight_icc_levels, axis=1)

# Save to Excel with openpyxl
styled.to_excel("radiomics_long_with_icc_highlighted.xlsx", index=False, engine='openpyxl')

# Download
from google.colab import files
files.download("radiomics_long_with_icc_highlighted.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
# Keep only rows with ICC ≥ 0.7
stable_data = melted_with_icc[melted_with_icc['ICC'] >= 0.7]

# Save just the stable feature measurements
stable_data.to_csv("stable_radiomics_data.csv", index=False)

# Download the file
from google.colab import files
files.download("stable_radiomics_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>